In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(1, '../')
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import transforms
from torch.utils.data import DataLoader

from src import *

In [ ]:
batch_size = 16
lr = 1e-5
latent_embedding = 512
df = pd.read_csv('../deepfake-detection/dfdc_train_preprocessed_20k/faces_metadata.csv')

In [ ]:
def train_test_split(df, fraction = 0.8):
    train_len = int(len(df)*0.8/2)
    train = pd.concat([df[df['label'] ==1 ].sample(frac=0.8, random_state=200), 
                       df[df['label'] ==0 ].sample(frac=0.8, random_state=200)])
    test = df.drop(train.index)
    return train.reindex(), test.reindex()

In [ ]:
train, test = train_test_split(df)
print(len(train), len(test))

In [ ]:
print(train.iloc[0])

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

trainset = DFDC_Dataset(df=train[:100], transform=transform, path='../deepfake-detection/dfdc_train_preprocessed_20k/')
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=16)
testset = DFDC_Dataset(df=test[:100], transform=transform, path='../deepfake-detection/dfdc_train_preprocessed_20k/')
testloader = DataLoader(testset, batch_size=batch_size, shuffle=True, num_workers=16)

In [ ]:
model = ConvLSTM(num_classes=1, attention=True, encoder='ResNet')
optim_, sched_ = CreateOptim(model.parameters(), lr=1e-5)

In [ ]:
losses = []
averages = []
train_dfd(model=model, dataloader=trainloader, testloader=testloader, optim=optim_, 
          scheduler=sched_, criterion=nn.BCELoss(),
          losses=losses, averages=averages, n_epochs=5, device = 'cuda:1', verbose=False)

In [ ]:
test_dfd(testloader, model, nn.BCELoss(), 'cuda:1')

In [ ]:
plt.plot(averages)

In [ ]:
save_checkpoint(model, '3 fps, 30 frames, 30 epochs', '../checkpoints/resnet152_attn_BCE.pth.tar')

In [ ]:
print(len(testset))

In [ ]:
a = torch.randn(16, 1)

In [ ]:
print(torch.round(a))